In [1]:
!pip install spacy google-cloud-storage google-cloud-bigquery
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import re
import spacy
import pandas as pd
from spacy.matcher import Matcher
from google.cloud import storage, bigquery
from datetime import datetime
import io

In [3]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define patterns for unwanted phrases
matcher = Matcher(nlp.vocab)
patterns = [
    [{"LOWER": "thank"}, {"LOWER": "you"}, {"LOWER": "for"}, {"LOWER": "your"}, {"LOWER": "query"}],
    [{"LOWER": "i"}, {"LOWER": "understand"}, {"LOWER": "your"}, {"LOWER": "concern"}],
    [{"LOWER": "sorry"}, {"IS_PUNCT": True}, {"LOWER": "the"}, {"LOWER": "images"}, {"LOWER": "were"}, {"LOWER": "not"}, {"LOWER": "clear"}],
    [{"LOWER": "attachment"}, {"LOWER": "removed"}, {"LOWER": "to"}, {"LOWER": "protect"}, {"LOWER": "patient"}, {"LOWER": "identity"}]
]
for pattern in patterns:
    matcher.add("UNWANTED_PHRASE", [pattern])

In [9]:
# List files in the GCS bucket
def list_gcs_files(bucket_name, prefix=""):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    return [blob.name for blob in bucket.list_blobs(prefix=prefix) if blob.name.endswith('.json')]

# Load JSON file from GCS
def load_json_from_gcs(bucket_name, file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    data = blob.download_as_text()
    return pd.read_json(io.StringIO(data), lines=True)

# Text cleaning functions
def remove_sentences_with_keywords(text, keywords=None):
    if keywords is None:
        keywords = ["Hello", "How are you doing", "patients identity", "welcome", "icliniq.com", "more details", "concern", "thank you", "picture"]
    pattern = r'(?i)\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return ' '.join([sentence for sentence in sentences if not re.search(pattern, sentence)])

def remove_unwanted_phrases(text):
    doc = nlp(text)
    matches = matcher(doc)
    spans = [doc[start:end] for match_id, start, end in matches]
    for span in spans:
        text = text.replace(span.text, "")
    return text.strip()

def clean_answer(text):
    text = remove_sentences_with_keywords(text)
    text = remove_unwanted_phrases(text)
    text = re.sub(r'\(attachment removed to protect the patient\'s identity\)', '', text, flags=re.IGNORECASE)
    if len(text.split()) < 20:
        return None
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^A-Za-z0-9.,? ]+', '', text)
    return text.strip()

# Process JSON files and load to BigQuery
def process_json_files(bucket_name, prefix, table_id):
    all_data = []
    file_names = list_gcs_files(bucket_name, prefix)

    if not file_names:
        print(f"No JSON files found in bucket {bucket_name} with prefix {prefix}")
        return

    for file_name in file_names:
        print(f"Processing file: {file_name}")
        try:
            df = load_json_from_gcs(bucket_name, file_name)
            if df.empty:
                print(f"File {file_name} is empty.")
                continue

            # Rename columns and apply cleaning
            df = df.rename(columns={"Doctor": "Answer", "Patient": "Question", "Description": "Title"})
            df['Answer'] = df['Answer'].apply(clean_answer)
            df['Title'] = df['Title'].str.replace(r'^Q\.\s*', '', regex=True)
            df = df.dropna(subset=['Answer']).reset_index(drop=True)

            # Add metadata columns
            df['source'] = 'mimic'
            df['last_updated_date'] = datetime.now().date()
            df['last_updated_date'] = pd.to_datetime(df['last_updated_date'], errors='coerce')

            if not df.empty:
                all_data.append(df[['Title', 'Question', 'Answer', 'source', 'last_updated_date']])
            else:
                print(f"No relevant data in file {file_name} after cleaning.")

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        load_to_bigquery(combined_df, table_id)
        print("All data loaded successfully.")
    else:
        print("No data to concatenate and upload to BigQuery.")

# Load to BigQuery
def load_to_bigquery(df, table_id):
    client = bigquery.Client()


    job = client.load_table_from_dataframe(df, table_id)
    job.result()
    print("Data loaded successfully into BigQuery.")




In [10]:
# Set GCS bucket, prefix, and BigQuery table details
bucket_name = "raw_dataset_genai"
prefix = "mimic/"
table_id = "health-ai-agent-sjsu.transformed_data.all_merged"

# Execute the pipeline
process_json_files(bucket_name, prefix, table_id)

Processing file: mimic/mimic_chunk_5_1.json
Data loaded successfully into BigQuery.
All data loaded successfully.
